# Comparing SGDOneClassSVM and OneClassSVM

This example compares the `sklearn.linear_model.SGDOneClassSVM` which implements a linear One-Class SVM using SGD, against `sklearn.svm.OneClassSVM` which implements a kernelized One-Class SVM.

Note that `sklearn.linear_model.SGDOneClassSVM` scales linearly with the number of samples whereas the complexity of a kernelized `sklearn.svm.OneClassSVM` is at best quadratic with respect to the number of samples. It is not the purpose of this example to illustrate the benefits of such an approximation in terms of computation time but rather to show that we obtain similar results on a toy dataset.

In [ ]:
# Authors: The scikit-learn developers
# License-Identifier: BSD-3-Clause

## Import required libraries

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.pyplot as plt
import numpy as np
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import SGDOneClassSVM
from sklearn.pipeline import make_pipeline
from sklearn.svm import OneClassSVM

## Set up plotting parameters

In [ ]:
font = {"weight": "normal", "size": 15}
plt.rc("font", **font)
random_state = 42
rng = np.random.RandomState(random_state)

## Generate synthetic data

We'll create regular data points for training and testing, plus some outlier points.

In [ ]:
# Generate some regular novel observations
X = 0.3 * rng.randn(500, 2)
X_train = np.r_[X + 2, X - 2]

# Generate some abnormal novel observations
X_test = np.r_[X + 2, X - 2]
X_outliers = rng.uniform(low=-4, high=4, size=(20, 2))

## Set hyperparameters and train the standard One-Class SVM

In [ ]:
# OCSVM hyperparameters
nu = 0.05
gamma = 2.0

# Fit the One-Class SVM model
clf = OneClassSVM(gamma=gamma, kernel="rbf", nu=nu)
clf.fit(X_train)

## Generate predictions with the standard One-Class SVM

In [ ]:
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)
y_pred_outliers = clf.predict(X_outliers)
n_error_train = y_pred_train[y_pred_train == -1].size
n_error_test = y_pred_test[y_pred_test == -1].size
n_error_outliers = y_pred_outliers[y_pred_outliers == 1].size

## Train the One-Class SVM with a kernel approximation and SGD

In [ ]:
# Fit the One-Class SVM using a kernel approximation and SGD
transform = Nystroem(gamma=gamma, random_state=random_state)
clf_sgd = SGDOneClassSVM(
    nu=nu, shuffle=True, fit_intercept=True, random_state=random_state, tol=1e-3
)
pipe_sgd = make_pipeline(transform, clf_sgd)
pipe_sgd.fit(X_train)

## Generate predictions with the SGD One-Class SVM

In [ ]:
y_pred_train_sgd = pipe_sgd.predict(X_train)
y_pred_test_sgd = pipe_sgd.predict(X_test)
y_pred_outliers_sgd = pipe_sgd.predict(X_outliers)
n_error_train_sgd = y_pred_train_sgd[y_pred_train_sgd == -1].size
n_error_test_sgd = y_pred_test_sgd[y_pred_test_sgd == -1].size
n_error_outliers_sgd = y_pred_outliers_sgd[y_pred_outliers_sgd == 1].size

## Create a visualization of the results

In [ ]:
# Create the figure
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# Set common parameters for both plots
titles = ["Kernel One-Class SVM", "Linear One-Class SVM with Nystroem kernel approx."]
models = [clf, pipe_sgd]
errors_train = [n_error_train, n_error_train_sgd]
errors_test = [n_error_test, n_error_test_sgd]
errors_outliers = [n_error_outliers, n_error_outliers_sgd]

# Plot results for both models
for i, (model, ax, title) in enumerate(zip(models, axs, titles)):
    # Plot the data points
    ax.scatter(X_train[:, 0], X_train[:, 1], c="white", edgecolors="k", s=40, label="training")
    ax.scatter(
        X_outliers[:, 0],
        X_outliers[:, 1],
        c="blueviolet",
        edgecolors="k",
        s=40,
        label="new regular",
    )

    # Create the decision boundary
    xx, yy = np.meshgrid(np.linspace(-5, 5, 100), np.linspace(-5, 5, 100))
    Z = model.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    # Plot the decision boundary and different regions
    ax.contourf(
        xx, yy, Z, levels=np.linspace(Z.min(), 0, 10), cmap=plt.cm.Blues_r, alpha=0.5
    )
    ax.contourf(
        xx, yy, Z, levels=[0, Z.max()], colors="palevioletred", alpha=0.5
    )
    ax.contour(xx, yy, Z, levels=[0], linewidths=2, colors="darkred")

    # Add stats as title
    ax.set_title(
        f"{title}\nTrain errors: {errors_train[i]}/1000; "
        f"Test errors: {errors_test[i]}/1000; "
        f"Outliers: {errors_outliers[i]}/20"
    )
    ax.set_xlim((-5, 5))
    ax.set_ylim((-5, 5))
    ax.set_xlabel("error (train)")
    ax.set_ylabel("error (test)")

plt.tight_layout()
plt.show()

## Compare summary statistics

In [ ]:
print("One-Class SVM with RBF kernel")
print(f"Training errors: {n_error_train} out of {X_train.shape[0]}")
print(f"Test errors: {n_error_test} out of {X_test.shape[0]}")
print(f"Outliers incorrectly classified: {n_error_outliers} out of {X_outliers.shape[0]}")
print("\nLinear One-Class SVM with Nystroem kernel approximation")
print(f"Training errors: {n_error_train_sgd} out of {X_train.shape[0]}")
print(f"Test errors: {n_error_test_sgd} out of {X_test.shape[0]}")
print(f"Outliers incorrectly classified: {n_error_outliers_sgd} out of {X_outliers.shape[0]}")